In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from datetime import datetime
csv_paths = [
    '../data/kaggle/data_by_artist.csv',
    '../data/kaggle/data_by_genres.csv',
    '../data/kaggle/data_by_year.csv',
    '../data/kaggle/data.csv',
    '../data/kaggle/data_w_genres.csv',
]
df_list = [pd.read_csv(path) for path in csv_paths]
a_track_id = '3jW3k9jZRHNZfpDxpbAsbn'

In [2]:
for i, df in enumerate(df_list):
    print(f"No. of duplicates \t{len(df) - len(df.drop_duplicates())}\tfor df{i}")
    if 'id' in df: print(f"No. of duplicates (id)\t{len(df) - len(df.drop_duplicates(subset=['id']))}\tfor df{i}")

df_by_artist = df_list[0]
df_by_genres = df_list[1]
df_by_year = df_list[2]
df_track = df_list[3].drop_duplicates(subset=['id'])  # Not a 100% sure why there are duplicates, can't be bothered to figure it out right now🙃
df_track['artists'] = df_track['artists'].apply(lambda x: ast.literal_eval(x))  # Artists is supposed to be a list in this csv
# handling edge cases pita just using year for now
# df_track['release_date'] = df_track['release_date'].apply(lambda x: (datetime.strptime(x, "%Y-%m-%d") if len(x.split('-')) == 3 else datetime.strptime(x, "%Y")))
df_w_genres = df_list[4]

def retrieve_track_attr_dict(id=None):
    if not id: row = df_track.sample(1).iloc[0]
    else: row = df_track.loc[df_track['id'] == id].iloc[0]  # removed duplicates so there is only 1 match
    return row.to_dict()

No. of duplicates 	0	for df0
No. of duplicates 	0	for df1
No. of duplicates 	0	for df2
No. of duplicates 	2159	for df3
No. of duplicates (id)	2159	for df3
No. of duplicates 	0	for df4


In [3]:
import Track, User


a_user = User()
print(a_user.get_optimal_track_pref())
a_user.update_track_attr_log(Track(retrieve_track_attr_dict()))
print(a_user.get_optimal_track_pref())
a_user.update_track_attr_log(Track(retrieve_track_attr_dict()))
print(a_user.get_optimal_track_pref())
a_user.update_track_attr_log(Track(retrieve_track_attr_dict()))
print(a_user.get_optimal_track_pref())

ImportError: attempted relative import with no known parent package